In [42]:
class Ticket:
    def __init__(self, id:int):
        self.id = id

    def __str__(self):
        return f"ticket id: {self.id}"

In [46]:
sorszam = Ticket(12)
print(sorszam)
assert sorszam.id == 12

ticket id: 12


In [63]:
class TicketMachine:
    tickets = list()
    
    def __str__(self):
        return f"tickets: {[ticket.__str__() for ticket in self.tickets]}"
    
    def getNewTicket(self):
        new_ticket = Ticket(len(self.tickets))
        self.tickets.append(new_ticket)
        return new_ticket

In [64]:
gep = TicketMachine()

In [68]:
sorszam = gep.getNewTicket()
print(sorszam)
print(f"A gepben levo sorszamok: {gep}")

ticket id: 3
A gepben levo sorszamok: tickets: ['ticket id: 0', 'ticket id: 1', 'ticket id: 2', 'ticket id: 3']


In [24]:
lista = [1, 2,  3]

[i*i for i in lista] # Ezt amugy hivjak list comprehension-nek a pythonosok

[1, 4, 9]

In [71]:
class Customer:
    def buyTicket(self, machine:TicketMachine):
        self.ticket = machine.getNewTicket()
        
    def showTicket(self):
        return self.ticket
    

In [72]:
bela_ba = Customer()
bela_ba.buyTicket(gep)
print(bela_ba.ticket)

ticket id: 4


In [77]:
class Staff:
    def __init__(self, machine:TicketMachine):
        self.machine = machine

    def validateTicket(self, ticket:Ticket):
        return ticket in self.machine.tickets

In [78]:
jolika = Staff(gep)


In [79]:
jolika.validateTicket(bela_ba.showTicket())

True